# SVM

In [2]:
import os 
import pandas as pd 
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, make_scorer, log_loss, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer

CLEAN_DATA_DIR = "../data/clean/"
RESULT_DATA_DIR = "../data/model_result/"

## Apply SVM on unfilled dataset
### Missing Values

In [ ]:
svm_unfilled = pd.read_csv(os.path.join(CLEAN_DATA_DIR, "TRAIN_MERGED_UNFILLED.csv"))

missing_columns = svm_unfilled.columns[svm_unfilled.isnull().any()].tolist()
print( 'Columns with na values are: ', missing_columns)

# Fill na values with 'missing'
svm_unfilled['Q06'] = svm_unfilled['Q06'].fillna(-1)
svm_unfilled['Q07'] = svm_unfilled['Q07'].fillna(-1)
svm_unfilled['Q08'] = svm_unfilled['Q08'].fillna(-1)
svm_unfilled['Q11'] = svm_unfilled['Q11'].fillna(-1)
svm_unfilled['Q19'] = svm_unfilled['Q19'].fillna(-1)

Columns with na values are:  ['Q06', 'Q07', 'Q08', 'Q11', 'Q19']


In [ ]:
# One-hot encode categorical columns
encoder = OneHotEncoder(sparse_output=False, drop=None)
encoded = encoder.fit_transform(svm_unfilled[missing_columns])


# Convert to DataFrame and combine with numerical features
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(missing_columns))
numerical_df = svm_unfilled.drop(columns=missing_columns)

# Combine numerical and encoded categorical data
processed_df = pd.concat([numerical_df, encoded_df], axis=1)

print("\nAfter One-Hot Encoding:")
print(processed_df)

### Divide data to train/test

In [ ]:
y = processed_df['subjectivePoverty_rating']
print(y)

feature_cols = list(processed_df.columns.difference(['psu_hh_idcode', 'hhid', 'subjectivePoverty_rating']))
X = processed_df[feature_cols]
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_x)
X_test_scaled = scaler.transform(test_x)

### Hyeprparameter tuning with grid search

In [ ]:
# GridSearch CV
param_grid = {
    'C': [0.5, 1, 10, 100],
    'gamma': ['scale', 1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

log_loss_scorer = make_scorer(log_loss, greater_is_better=False, needs_proba=True)
optimal_params = GridSearchCV(SVC(probability=True, random_state=42), param_grid, cv=5, scoring='neg_log_loss', verbose=2)
optimal_params.fit(X_train_scaled, train_y)
print("Best Parameters:", optimal_params.best_params_)
print("Best Log Loss:", optimal_params.best_score_)



# Best Parameters: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
# Best Log Loss: -1.948786842738619


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END .....................C=0.5, gamma=scale, kernel=rbf; total time=   2.4s
[CV] END .....................C=0.5, gamma=scale, kernel=rbf; total time=   2.3s
[CV] END .....................C=0.5, gamma=scale, kernel=rbf; total time=   2.3s
[CV] END .....................C=0.5, gamma=scale, kernel=rbf; total time=   2.3s
[CV] END .....................C=0.5, gamma=scale, kernel=rbf; total time=   2.3s
[CV] END .........................C=0.5, gamma=1, kernel=rbf; total time=   2.8s
[CV] END .........................C=0.5, gamma=1, kernel=rbf; total time=   2.8s
[CV] END .........................C=0.5, gamma=1, kernel=rbf; total time=   3.0s
[CV] END .........................C=0.5, gamma=1, kernel=rbf; total time=   3.0s
[CV] END .........................C=0.5, gamma=1, kernel=rbf; total time=   3.0s
[CV] END .......................C=0.5, gamma=0.1, kernel=rbf; total time=   2.8s
[CV] END .......................C=0.5, gamma=0.

In [ ]:
# Best Parameters: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
# Best Log Loss: -1.948546531080508
svm_model = SVC(C=10, gamma=0.01, kernel='rbf', probability=True, random_state=42)
svm_model.fit(X_train_scaled, train_y)
pred = svm_model.predict_proba(X_test_scaled)
log_loss(test_y, pred)
#1.9642149345026059

X_test_scaled.size()